In [ ]:
# Imports

import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
from glob import glob
from matplotlib import pyplot as plt
from datetime import datetime

In [ ]:
# Directory Path:
IMG_PATH = "../data/pets/train/"
LABEL_PATH = "../data/pets/train_label.txt"

# Constants
NUM_IMG = 2000
N_CLUSTER = 200 # K-means 

In [ ]:
# Image Processing & General Feature Extraction

# Gray scale image
def gray(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# SIFT feature
def get_features(image, SIFT_obj):
    keypoints, features = SIFT_obj.detectAndCompute(image, None) # Don't need grayscale image here
    return keypoints, features


In [ ]:
# Deeper Feature Extraction

# Bag of word
# clustering
def bow_cluster(kmeans_obj, descriptor_stack):
    kmeans_ret = kmeans_obj.fit_predict(descriptor_stack)
    return kmeans_ret

# generate vertical stack of descriptors
def bow_vstack(desc_list):
    stack = np.array(desc_list[0])
    for rest in desc_list[1:]:
        stack = np.vstack((stack, rest))
    desc_stack = stack.copy()
    return desc_stack

# generate bag of words frequency matrix (shape: NUM_TRAIN_IMG * N_CLUSTER)
def bow_get_freq_matrix(num_imgs, num_clusters, SIFT_list, kmeans_ret):
    # initialization
    matrix = np.array([np.zeros(num_clusters) for i in range(num_imgs)])
    
    # keep track of index of kmeans_ret
    kmeans_id = 0
    for i in range(num_imgs):
        l = len(SIFT_list[i])
        for j in range(l):
            cluster_id = kmeans_ret[kmeans_id + j]
            matrix[i][cluster_id] += 1
        kmeans_id += l
    
    return matrix
            

In [ ]:
# Read In Image Files
def get_images(path):
    imlist = []
    for i in range(NUM_IMG):
        path_str = path + "pet" + str(i+1) + ".jpg"
        im = cv2.imread(path_str)
        imlist.append(im)
        
    return imlist

In [ ]:
start_time = datetime.now()
print("Reading Images @ " + str(start_time))
imlist = get_images(IMG_PATH) # imlist contains a list of image numpy arrays
end_time = datetime.now()
print("Complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

In [ ]:
print(len(imlist))

In [ ]:
# Read In Labels
def get_labels(path):
    labels = np.loadtxt(path, dtype = 'str')
    np.reshape(labels, (-1, 1))
    return labels

In [ ]:
labellist = get_labels(LABEL_PATH)

In [ ]:
print(labellist.shape)

In [ ]:
## Compute SIFT features
start_time = datetime.now()
print("Generating SIFT features @ " + str(start_time))
SIFT_obj = cv2.xfeatures2d.SIFT_create()
SIFT_list = []
for im in imlist:
    keypoint, descriptor = get_features(im, SIFT_obj)
    SIFT_list.append(descriptor)
end_time = datetime.now()
print("SIFT features complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

In [ ]:
# Generate a vertical stack of descriptors to perform clustering
start_time = datetime.now()
print("Generating SIFT stacked matrix @ " + str(start_time))
descriptor_stack = bow_vstack(SIFT_list)

end_time = datetime.now()
print("Matrix complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))
print("Shape of vstack matrix: " + str(descriptor_stack.shape))

In [ ]:
# BOW clustering
start_time = datetime.now()
print("Start Clustering @ " + str(start_time))

kmeans_obj = KMeans(n_clusters = N_CLUSTER)
kmeans_ret = bow_cluster(kmeans_obj, descriptor_stack)

end_time = datetime.now()
print("Matrix complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))
print("Shape of kmeans ret: " + str(kmeans_ret.shape))

In [ ]:
start_time = datetime.now()
print("Generating BOW Vocabulary @ " + str(start_time))

vocab_matrix = bow_get_freq_matrix(NUM_IMG, N_CLUSTER, SIFT_list, kmeans_ret)

#### IMPORTANT: FOR SVM, STANDARDIZE DATA BEFORE FEEDING INTO SVC()
scale = StandardScaler().fit(vocab_matrix)
vocab_matrix_std = scale.transform(vocab_matrix)

end_time = datetime.now()
print("BOW Vocabulary complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))
print("Plot Histogram of entire dataset (x = vocabulary, y = count)")
x_scalar = np.arange(N_CLUSTER)
y_scalar = np.array([abs(np.sum(vocab_matrix[:, h], dtype = np.int32)) for h in range(N_CLUSTER)])
plt.bar(x_scalar, y_scalar)
plt.xlabel("Vocabulary Index")
plt.ylabel("Frequency")
plt.title("BOW frequency histogram")
plt.xticks(x_scalar + 0.4, x_scalar)
plt.show()

In [ ]:
print(vocab_matrix.shape)

In [ ]:
train_labels = np.asarray(labellist)
print(train_labels.shape)
print(vocab_matrix_std.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vocab_matrix_std, train_labels, test_size = 0.2, random_state =42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
start_time = datetime.now()
print("Start training using SVM @ " + str(start_time))

clf = SVC()
clf.fit(X_train, y_train)
pred = np.asarray(clf.predict(X_test))
accuracy = (pred == y_test).mean()
print("Accuracy: %.2f%%" % (accuracy * 100))
end_time = datetime.now()
print("End training @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

In [ ]:
# Save vocabulary feature matrix to output folder in csv format and npy format
from pandas import DataFrame
df = DataFrame(data = vocab_matrix)
df.index.name = 'IMG_ID'
column_name_list = []
for i in range(N_CLUSTER):
    column_name_list.append('CLUSTER_ID_' + str(i))
df.columns = column_name_list
df.to_csv('../output/BOWmatrix-' + str(N_CLUSTER) + '.csv', mode = 'a', index = True, sep = ',')
np.save('../output/BOWmatrix-' + str(N_CLUSTER) + '.npy', df)



In [ ]:
df2 = DataFrame(data = vocab_matrix_std)
df2.index.name = 'IMG_ID'
df2.columns = column_name_list
df2.to_csv('../output/BOWmatrix_std-' + str(N_CLUSTER) + '.csv', mode = 'a', index = True, sep = ',')
np.save('../output/BOWmatrix_std' + str(N_CLUSTER) + '.npy', df2)

In [ ]:
from sklearn.externals import joblib
kmeans_filename = '../output/kmeans' + str(N_CLUSTER) + '.sav'
print("Saving kmeans model")
joblib.dump(kmeans_obj, kmeans_filename)
print("Saving complete")

In [ ]:
# Test saving
model = joblib.load(kmeans_filename)
SIFT_obj_tmp = cv2.xfeatures2d.SIFT_create()
kp, des = get_features(imlist[0], SIFT_obj_tmp)
print("Shape of descriptor: " + str(des.shape))
vocab = np.array( [[ 0 for i in range(N_CLUSTER)]])
test_ret = model.predict(des)
print("Shape of ret: " + str(test_ret.shape))
for i in test_ret:
    vocab[0][i] += 1
vocab = scale.transform(vocab)
pred_test = clf.predict(vocab)
print(pred_test)